In [16]:
# Install required dependencies
%pip install shapely -q
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import json
from pathlib import Path
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from dotenv import load_dotenv
from rosbags.rosbag2 import Reader
from rosbags.typesys import Stores, get_typestore

# Load environment variables
PARENT_ENV = Path(__file__).resolve().parents[1] / ".env" if '__file__' in globals() else Path.cwd().parent / ".env"
load_dotenv(dotenv_path=PARENT_ENV)

# Define paths
ROSBAGS_DIR_NAS = os.getenv("ROSBAGS_DIR_NAS")
IMAGES_PER_TOPIC_DIR = os.getenv("IMAGES_PER_TOPIC_DIR")
GNSS_FILTER = "/mnt/data/gps_filter/polygons/oberholz.json"
POLYGON_BUFFER_METERS = 30  # Shrink polygon by 30 meters inward for safety
OUTPUT_DIR = Path("/mnt/data/gnss_aligned_data")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Create typestore for message deserialization
typestore = get_typestore(Stores.LATEST)

print(f"ROSBAGS_DIR_NAS: {ROSBAGS_DIR_NAS}")
print(f"IMAGES_PER_TOPIC_DIR: {IMAGES_PER_TOPIC_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")


ROSBAGS_DIR_NAS: /home/nepomuk/sflnas/DataReadOnly334/tractor_data/autorecord
IMAGES_PER_TOPIC_DIR: /home/nepomuk/sflnas/DataReadWrite334/0_shared/Feldschwarm/bagseek/src/extracted_images_per_topic
OUTPUT_DIR: /mnt/data/gnss_aligned_data


In [3]:
# GPS Polygon Filter

def load_gps_polygon(polygon_file):
    """Load GPS polygon from JSON file."""
    try:
        with open(polygon_file, 'r') as f:
            data = json.load(f)
        
        # Extract coordinates - handle multiple formats
        if "coordinates" in data:
            # GeoJSON format: {"coordinates": [[[lon, lat], [lon, lat], ...]] }
            coords = data["coordinates"]
            # Handle nested lists (GeoJSON MultiPolygon or Polygon)
            if isinstance(coords[0][0], list):
                # Nested: take first polygon
                coords = coords[0]
            # Convert [lon, lat] to (lat, lon)
            polygon = [(coord[1], coord[0]) for coord in coords]
        elif "polygon" in data:
            # Simple format: [{"lat": ..., "lon": ...}, ...]
            polygon = [(p["lat"], p["lon"]) for p in data["polygon"]]
        else:
            # Try to parse as list directly
            polygon = data if isinstance(data, list) else []
        
        print(f"✓ Loaded GPS polygon with {len(polygon)} points")
        return polygon
    except Exception as e:
        print(f"⚠ Warning: Could not load GPS polygon: {e}")
        return None

def point_in_polygon(lat, lon, polygons):
    """
    Check if a point (lat, lon) is inside any of the polygons using ray-casting algorithm.
    
    Args:
        lat: Latitude of the point
        lon: Longitude of the point
        polygons: List of polygons, where each polygon is a list of (lat, lon) tuples
    
    Returns:
        bool: True if point is inside any polygon, False otherwise
    """
    # Handle both single polygon and list of polygons
    if not polygons:
        return False
    
    # If it's a single polygon (list of tuples), wrap it in a list
    if len(polygons) > 0 and isinstance(polygons[0], tuple) and len(polygons[0]) == 2:
        polygons = [polygons]
    
    # Check if point is inside ANY of the polygons
    for polygon in polygons:
        if not polygon or len(polygon) < 3:
            continue
        
        n = len(polygon)
        inside = False
        
        p1_lat, p1_lon = polygon[0]
        for i in range(1, n + 1):
            p2_lat, p2_lon = polygon[i % n]
            
            # Check if point is on an edge (ray crosses the edge)
            if lon > min(p1_lon, p2_lon):
                if lon <= max(p1_lon, p2_lon):
                    if lat <= max(p1_lat, p2_lat):
                        if p1_lon != p2_lon:
                            x_intersection = (lon - p1_lon) * (p2_lat - p1_lat) / (p2_lon - p1_lon) + p1_lat
                        if p1_lat == p2_lat or lat <= x_intersection:
                            inside = not inside
            
            p1_lat, p1_lon = p2_lat, p2_lon
        
        # If point is inside this polygon, return True immediately
        if inside:
            return True
    
    # Not inside any polygon
    return False

def buffer_polygon_inward(polygon, buffer_meters):
    """
    Shrink polygon inward by specified meters using shapely.
    Can return multiple polygons if buffer splits the original polygon.
    
    Args:
        polygon: List of (lat, lon) tuples
        buffer_meters: Distance in meters to shrink inward (positive = shrink, negative = expand)
    
    Returns:
        List of polygons, where each polygon is a list of (lat, lon) tuples
    """
    try:
        from shapely.geometry import Polygon
        
        # Convert to (lon, lat) for shapely
        coords_lonlat = [(lon, lat) for lat, lon in polygon]
        poly = Polygon(coords_lonlat)
        
        # Approximate conversion: meters to degrees
        # At latitude ~51°: 1 degree lat ≈ 111km, 1 degree lon ≈ 70km
        avg_lat = sum(lat for lat, lon in polygon) / len(polygon)
        meters_per_degree_lat = 111000
        meters_per_degree_lon = 111000 * np.cos(np.radians(avg_lat))
        
        # Convert meters to degrees (negative to shrink inward)
        buffer_degrees = -buffer_meters / ((meters_per_degree_lat + meters_per_degree_lon) / 2)
        
        # Apply buffer
        buffered_poly = poly.buffer(buffer_degrees)
        
        # Convert back to (lat, lon) list
        if buffered_poly.is_empty:
            print(f"⚠ Warning: Buffer of {buffer_meters}m too large, polygon disappeared. Using original.")
            return [polygon]
        
        # Handle MultiPolygon (keep ALL polygons, not just the largest)
        polygons = []
        if buffered_poly.geom_type == 'MultiPolygon':
            # Keep all polygon components
            for poly_component in buffered_poly.geoms:
                buffered_coords = list(poly_component.exterior.coords)
                polygons.append([(lat, lon) for lon, lat in buffered_coords])
            print(f"  Note: Buffer split polygon into {len(polygons)} separate areas")
        else:
            # Single Polygon
            buffered_coords = list(buffered_poly.exterior.coords)
            polygons.append([(lat, lon) for lon, lat in buffered_coords])
        
        return polygons
        
    except ImportError:
        print("⚠ shapely not installed. Install it to use polygon buffering")
        print("  Using original polygon without buffer")
        return [polygon]
    except Exception as e:
        print(f"⚠ Error applying buffer: {e}")
        return [polygon]

# Load polygon if file exists
GPS_POLYGON = None
if Path(GNSS_FILTER).exists():
    original_polygon = load_gps_polygon(Path(GNSS_FILTER))
    
    # Apply inward buffer if configured
    if original_polygon and POLYGON_BUFFER_METERS > 0:
        original_points = len(original_polygon)
        GPS_POLYGON = buffer_polygon_inward(original_polygon, POLYGON_BUFFER_METERS)
        total_points = sum(len(poly) for poly in GPS_POLYGON)
        print(f"✓ Applied {POLYGON_BUFFER_METERS}m inward buffer ({original_points} pts → {len(GPS_POLYGON)} polygon(s), {total_points} total pts)")
    else:
        # No buffer, wrap single polygon in list
        GPS_POLYGON = [original_polygon] if original_polygon else None
else:
    print(f"⚠ GPS polygon file not found: {GNSS_FILTER}")
    print("  Records will not be filtered by location.")


✓ Loaded GPS polygon with 0 points


## GPS Polygon Format

The `gps_polygon.json` file should contain a polygon defining the allowed area. Supported formats:

**Format 1 - GeoJSON style:**
```json
{
  "coordinates": [
    [longitude1, latitude1],
    [longitude2, latitude2],
    [longitude3, latitude3]
  ]
}
```

**Format 2 - Simple style:**
```json
{
  "polygon": [
    {"lat": latitude1, "lon": longitude1},
    {"lat": latitude2, "lon": longitude2},
    {"lat": latitude3, "lon": longitude3}
  ]
}
```

Each record in the output JSONL will have an `"in_area": true/false` field indicating if the GPS coordinates are within this polygon.

## Safety Buffer

The `POLYGON_BUFFER_METERS` variable (default: 30 meters) shrinks the polygon inward to ensure the camera stays away from edges, streets, etc.

- Set to `0` to disable buffering
- Set to `30` (default) for 30-meter safety margin
- Set to higher values for more conservative filtering

The buffer is applied automatically when loading the polygon.


In [4]:
# Alignment functions (from 01_generate_alignment_and_metadata.py)

def determine_reference_topic(topic_timestamps):
    """Determine the topic with the most messages to use as the reference timeline."""
    return max(topic_timestamps.items(), key=lambda x: len(x[1]))[0]

def create_reference_timestamps(timestamps, factor=2):
    """Create a refined reference timeline with smaller intervals to improve alignment accuracy."""
    timestamps = sorted(set(timestamps))
    diffs = np.diff(timestamps)
    mean_interval = np.mean(diffs)
    refined_interval = mean_interval / factor
    ref_start = timestamps[0]
    ref_end = timestamps[-1]
    return np.arange(ref_start, ref_end, refined_interval).astype(np.int64)

def align_topic_to_reference(topic_ts, ref_ts, max_diff=int(1e8)):
    """Align timestamps of a topic to the closest reference timestamps within max_diff."""
    aligned = []
    topic_idx = 0
    for rt in ref_ts:
        while topic_idx + 1 < len(topic_ts) and abs(topic_ts[topic_idx + 1] - rt) < abs(topic_ts[topic_idx] - rt):
            topic_idx += 1
        closest = topic_ts[topic_idx]
        if abs(closest - rt) <= max_diff:
            aligned.append(closest)
        else:
            aligned.append(None)
    return aligned

def topic_to_directory_name(topic: str) -> str:
    """Convert topic name to directory-safe name."""
    return topic.replace("/", "__")

print("✓ Alignment functions loaded")


✓ Alignment functions loaded


In [5]:
# Message data extraction functions

def extract_gnss_data(msg):
    """Extract GNSS/GPS data from NavSatFix message."""
    try:
        return {
            "latitude": float(msg.latitude),
            "longitude": float(msg.longitude),
            "altitude": float(msg.altitude),
            "status": int(msg.status.status) if hasattr(msg, 'status') else None,
            "service": int(msg.status.service) if hasattr(msg, 'status') else None,
        }
    except Exception as e:
        return {"error": str(e)}

def extract_tf_data(msg):
    """Extract TF (Transform) data from TFMessage."""
    try:
        transforms = []
        for transform in msg.transforms:
            tf_data = {
                "child_frame_id": transform.child_frame_id,
                "header": {
                    "frame_id": transform.header.frame_id,
                    "stamp": {
                        "sec": int(transform.header.stamp.sec),
                        "nanosec": int(transform.header.stamp.nanosec)
                    }
                },
                "transform": {
                    "translation": {
                        "x": float(transform.transform.translation.x),
                        "y": float(transform.transform.translation.y),
                        "z": float(transform.transform.translation.z)
                    },
                    "rotation": {
                        "x": float(transform.transform.rotation.x),
                        "y": float(transform.transform.rotation.y),
                        "z": float(transform.transform.rotation.z),
                        "w": float(transform.transform.rotation.w)
                    }
                }
            }
            transforms.append(tf_data)
        return {"transforms": transforms}
    except Exception as e:
        return {"error": str(e)}

def get_image_path(rosbag_name, topic, timestamp):
    """Get the relative path to the extracted image file."""
    topic_dir = topic_to_directory_name(topic)
    # Return relative path: extracted_images_per_topic/rosbag_name/topic_dir/timestamp.webp
    relative_path = f"extracted_images_per_topic/{rosbag_name}/{topic_dir}/{timestamp}.webp"
    
    # Check if file actually exists (using absolute path)
    absolute_path = Path(IMAGES_PER_TOPIC_DIR) / rosbag_name / topic_dir / f"{timestamp}.webp"
    if absolute_path.exists():
        return relative_path
    else:
        return None

print("✓ Data extraction functions loaded")


✓ Data extraction functions loaded


In [6]:
def classify_topic_type(topic_name, msg_type):
    """Classify topic as Image, GNSS, or TF based on message type."""
    # Only accept the front left camera
    if "Image" in msg_type:
        # Filter for only the front left camera
        if "zed_node" in topic_name and "left" in topic_name:
            return "image"
        return None  # Reject other image topics
    elif "NavSatFix" in msg_type or "GPS" in msg_type or "GNSS" in msg_type:
        return "gnss"
    elif topic_name == "/tf" or "TFMessage" in msg_type:
        return "tf"
    return None

def process_rosbag_with_data(rosbag_path, output_jsonl_path):
    """
    Process rosbag and extract aligned data for images, GNSS, and IMU.
    
    Args:
        rosbag_path: Path to the rosbag directory
        output_jsonl_path: Path to output JSONL file
    """
    rosbag_name = os.path.basename(rosbag_path)
    print(f"\n{'='*80}")
    print(f"Processing: {rosbag_name}")
    print(f"{'='*80}")
    
    # Store timestamps and messages by topic
    topic_data = defaultdict(list)
    topic_messages = defaultdict(dict)  # topic -> {timestamp: message_data}
    topic_types = {}
    topic_classifications = {}
    
    # Read all messages from rosbag
    print("Reading rosbag...")
    try:
        with Reader(rosbag_path) as reader:
            for connection, timestamp, rawdata in tqdm(reader.messages(), desc="Reading messages"):
                topic = connection.topic
                msg_type = connection.msgtype
                
                # Store message type
                if topic not in topic_types:
                    topic_types[topic] = msg_type
                    topic_classifications[topic] = classify_topic_type(topic, msg_type)
                
                # Only process image, GNSS, and IMU topics
                topic_class = topic_classifications[topic]
                if topic_class is None:
                    continue
                
                # Store timestamp
                topic_data[topic].append(timestamp)
                
                # Deserialize and extract data based on topic type
                try:
                    msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
                    
                    if topic_class == "gnss":
                        topic_messages[topic][timestamp] = extract_gnss_data(msg)
                    elif topic_class == "tf":
                        topic_messages[topic][timestamp] = extract_tf_data(msg)
                    # For images, we just store the timestamp (path is computed later)
                    
                except Exception as e:
                    # Skip messages that fail to deserialize
                    continue
    
    except Exception as e:
        print(f"Error reading rosbag: {e}")
        return
    
    if not topic_data:
        print("No relevant topics found in rosbag")
        return
    
    # Separate topics by classification
    image_topics = [t for t, c in topic_classifications.items() if c == "image"]
    gnss_topics = [t for t, c in topic_classifications.items() if c == "gnss"]
    tf_topics = [t for t, c in topic_classifications.items() if c == "tf"]
    
    print(f"\nFound:")
    print(f"  Image topics: {len(image_topics)}")
    print(f"  GNSS topics: {len(gnss_topics)}")
    print(f"  TF topics: {len(tf_topics)}")
    
    # Determine reference topic (use topic with most messages)
    ref_topic = determine_reference_topic(topic_data)
    print(f"\nReference topic: {ref_topic} ({len(topic_data[ref_topic])} messages)")
    
    # Create reference timeline
    print("Creating reference timeline...")
    ref_ts = create_reference_timestamps(topic_data[ref_topic])
    print(f"Reference timeline: {len(ref_ts)} timestamps")
    
    # Align all topics to reference
    print("\nAligning topics to reference timeline...")
    aligned_data = {}
    for topic, timestamps in tqdm(topic_data.items(), desc="Aligning topics"):
        aligned_data[topic] = align_topic_to_reference(sorted(timestamps), ref_ts)
    
    print(f"\n✓ Alignment complete")
    return ref_ts, aligned_data, topic_classifications, topic_messages, rosbag_name

print("✓ Main processing function loaded")


✓ Main processing function loaded


In [7]:
def write_aligned_jsonl(ref_ts, aligned_data, topic_classifications, topic_messages, rosbag_name, output_path):
    """
    Write aligned data to JSONL file.
    
    Each line in the JSONL file represents one reference timestamp with all aligned data.
    """
    print(f"\nWriting aligned data to {output_path}...")
    
    # Separate topics by type
    image_topics = [t for t, c in topic_classifications.items() if c == "image"]
    gnss_topics = [t for t, c in topic_classifications.items() if c == "gnss"]
    tf_topics = [t for t, c in topic_classifications.items() if c == "tf"]
    
    records_written = 0
    
    with open(output_path, 'w') as f:
        for i, ref_time in enumerate(tqdm(ref_ts, desc="Writing JSONL")):
            record = {
                "reference_timestamp": int(ref_time),
                "images": {},
                "gnss": {},
                "tf": {}
            }
            
            # Add aligned image data
            for topic in image_topics:
                aligned_ts = aligned_data[topic][i]
                if aligned_ts is not None:
                    image_path = get_image_path(rosbag_name, topic, aligned_ts)
                    record["images"][topic] = {
                        "timestamp": int(aligned_ts),
                        "path": image_path
                    }
            
            # Add aligned GNSS data
            for topic in gnss_topics:
                aligned_ts = aligned_data[topic][i]
                if aligned_ts is not None:
                    gnss_data = topic_messages[topic].get(aligned_ts)
                    if gnss_data:
                        record["gnss"][topic] = {
                            "timestamp": int(aligned_ts),
                            "data": gnss_data
                        }
            
            # Add aligned TF data
            for topic in tf_topics:
                aligned_ts = aligned_data[topic][i]
                if aligned_ts is not None:
                    tf_data = topic_messages[topic].get(aligned_ts)
                    if tf_data:
                        record["tf"][topic] = {
                            "timestamp": int(aligned_ts),
                            "data": tf_data
                        }
            
            # Check if location is within polygon (using first GNSS topic if available)
            in_area = False
            if GPS_POLYGON and record["gnss"]:
                # Get first GNSS topic data
                first_gnss = next(iter(record["gnss"].values()), None)
                if first_gnss and "data" in first_gnss:
                    gnss_data = first_gnss["data"]
                    if "latitude" in gnss_data and "longitude" in gnss_data:
                        lat = gnss_data["latitude"]
                        lon = gnss_data["longitude"]
                        in_area = point_in_polygon(lat, lon, GPS_POLYGON)
            
            record["in_area"] = in_area
            
            # Write record as JSON line
            f.write(json.dumps(record) + '\n')
            records_written += 1
    
    print(f"✓ Wrote {records_written} aligned records to {output_path}")
    return records_written

print("✓ JSONL writer function loaded")


✓ JSONL writer function loaded


## Example Usage

Process a single rosbag and generate aligned JSONL data:


In [8]:
# Example: Process a single rosbag
# Change this to the rosbag you want to process

#rosbag_name = "rosbag2_2025_09_25-08_05_56"  # Change this to your rosbag
#rosbag_path = os.path.join(ROSBAGS_DIR_NAS, rosbag_name)
rosbag_path = "/home/nepomuk/sflnas/DataReadOnly334/tractor_data/autorecord/rosbag2_2025_09_24-14_13_40"
rosbag_name = os.path.basename(rosbag_path)
output_jsonl = OUTPUT_DIR / f"{rosbag_name}_aligned.jsonl"

if os.path.exists(rosbag_path):
    result = process_rosbag_with_data(rosbag_path, output_jsonl)
    
    if result:
        ref_ts, aligned_data, topic_classifications, topic_messages, bag_name = result
        records_written = write_aligned_jsonl(
            ref_ts, aligned_data, topic_classifications, 
            topic_messages, bag_name, output_jsonl
        )
        print(f"\n✅ Success! Aligned data saved to: {output_jsonl}")
    else:
        print("❌ Failed to process rosbag")
else:
    print(f"❌ Rosbag not found: {rosbag_path}")



Processing: rosbag2_2025_09_24-14_13_40
Reading rosbag...


Reading messages: 10581it [00:14, 749.22it/s]



Found:
  Image topics: 1
  GNSS topics: 1
  TF topics: 2

Reference topic: /tf (3303 messages)
Creating reference timeline...
Reference timeline: 6605 timestamps

Aligning topics to reference timeline...


Aligning topics: 100%|██████████| 4/4 [00:00<00:00, 484.55it/s]



✓ Alignment complete

Writing aligned data to /mnt/data/gnss_aligned_data/rosbag2_2025_09_24-14_13_40_aligned.jsonl...


Writing JSONL: 100%|██████████| 6605/6605 [00:00<00:00, 44083.33it/s]

✓ Wrote 6605 aligned records to /mnt/data/gnss_aligned_data/rosbag2_2025_09_24-14_13_40_aligned.jsonl

✅ Success! Aligned data saved to: /mnt/data/gnss_aligned_data/rosbag2_2025_09_24-14_13_40_aligned.jsonl


## Preview Output

View the first few records from the generated JSONL file:


In [9]:
# Test the polygon with a known point
test_lat = 51.256709
test_lon = 12.520784

if GPS_POLYGON:
    # Check if point is in any of the polygons
    in_polygon = point_in_polygon(test_lat, test_lon, GPS_POLYGON)
    print(f"Test point: lat={test_lat}, lon={test_lon}")
    print(f"Point in polygon: {in_polygon}")
    
    # Handle multiple polygons
    if isinstance(GPS_POLYGON[0], tuple):
        # Single polygon
        polygons = [GPS_POLYGON]
    else:
        # Multiple polygons
        polygons = GPS_POLYGON
    
    print(f"\nPolygon bounds:")
    print("-" * 75)
    
    # Flatten all coordinates from all polygons
    all_coords = []
    for polygon in polygons:
        for lat, lon in polygon:
            all_coords.append((lat, lon))
    
    all_lats = [coord[0] for coord in all_coords]
    all_lons = [coord[1] for coord in all_coords]
    
    print(f"  Latitude range: {min(all_lats):.6f} to {max(all_lats):.6f}")
    print(f"  Longitude range: {min(all_lons):.6f} to {max(all_lons):.6f}")
    print(f"\nPolygon points (first 5):")
    for i, (lat, lon) in enumerate(all_coords[:5]):
        print(f"  {i+1}: lat={lat:.6f}, lon={lon:.6f}")
    
    print(f"\nTotal points: {len(all_coords)}")
    print(f"Number of polygons: {len(polygons)}")
else:
    print("No GPS polygon loaded")

No GPS polygon loaded


In [10]:
# Visualization: Plot polygon and GPS path
import matplotlib.pyplot as plt

def plot_polygon_and_path(polygons, jsonl_path, max_points=None):
    """
    Plot the GPS polygon(s) and the rosbag's GPS path.
    
    Args:
        polygons: List of polygons, where each polygon is a list of (lat, lon) tuples
        jsonl_path: Path to the JSONL file with GPS data
        max_points: Maximum number of GPS points to plot (for performance)
    """
    if not polygons:
        print("No polygons loaded")
        return
    
    # Handle single polygon wrapped in list
    if isinstance(polygons[0], tuple):
        polygons = [polygons]
    
    # Extract GPS points from JSONL
    gps_points_inside = []
    gps_points_outside = []
    
    with open(jsonl_path, 'r') as f:
        for i, line in enumerate(f):
            if max_points is not None and i >= max_points:
                break
            record = json.loads(line)
            if record.get("gnss"):
                first_gnss = next(iter(record["gnss"].values()), None)
                if first_gnss and "data" in first_gnss:
                    lat = first_gnss["data"]["latitude"]
                    lon = first_gnss["data"]["longitude"]
                    
                    if record.get("in_area"):
                        gps_points_inside.append((lat, lon))
                    else:
                        gps_points_outside.append((lat, lon))
    
    # Create plot
    plt.figure(figsize=(12, 10))
    
    # Plot all polygons
    for idx, polygon in enumerate(polygons):
        poly_lats = [p[0] for p in polygon]
        poly_lons = [p[1] for p in polygon]
        
        # Close the polygon for plotting
        poly_lats_closed = poly_lats + [poly_lats[0]]
        poly_lons_closed = poly_lons + [poly_lons[0]]
        
        label = f'Polygon {idx+1}' if len(polygons) > 1 else 'Polygon boundary'
        plt.plot(poly_lons_closed, poly_lats_closed, 'b-', linewidth=2, label=label if idx == 0 or len(polygons) > 1 else None)
        plt.fill(poly_lons_closed, poly_lats_closed, alpha=0.1, color='blue')
    
    # Plot GPS points inside polygon
    if gps_points_inside:
        inside_lats = [p[0] for p in gps_points_inside]
        inside_lons = [p[1] for p in gps_points_inside]
        plt.scatter(inside_lons, inside_lats, c='green', s=10, alpha=0.5, label=f'Inside ({len(gps_points_inside)} points)')
    
    # Plot GPS points outside polygon
    if gps_points_outside:
        outside_lats = [p[0] for p in gps_points_outside]
        outside_lons = [p[1] for p in gps_points_outside]
        plt.scatter(outside_lons, outside_lats, c='red', s=10, alpha=0.5, label=f'Outside ({len(gps_points_outside)} points)')
    
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('GPS Polygon and Rosbag Path')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
    print(f"\nStatistics:")
    print(f"  Points inside polygon: {len(gps_points_inside)}")
    print(f"  Points outside polygon: {len(gps_points_outside)}")
    print(f"  Total points plotted: {len(gps_points_inside) + len(gps_points_outside)}")

print("✓ Visualization function loaded")


✓ Visualization function loaded


In [11]:
# Visualize polygon and rosbag GPS path (ALL points)
if GPS_POLYGON and output_jsonl.exists():
    plot_polygon_and_path(GPS_POLYGON, output_jsonl, max_points=None)
else:
    if not GPS_POLYGON:
        print("❌ GPS polygon not loaded")
    if not output_jsonl.exists():
        print(f"❌ JSONL file not found: {output_jsonl}")


❌ GPS polygon not loaded


## Interactive Map (Optional)

For an interactive map with zoom/pan capabilities, install folium: `pip install folium`

Then run the cell below:


In [12]:
# Interactive map with folium (optional - requires: pip install folium)
try:
    import folium
    from folium import plugins
    
    if GPS_POLYGON and output_jsonl.exists():
        # Handle single polygon or list of polygons
        polygons = GPS_POLYGON if isinstance(GPS_POLYGON, list) else [GPS_POLYGON]
        if isinstance(polygons[0], tuple):
            polygons = [polygons]
        
        # Calculate center from all polygons
        all_lats = []
        all_lons = []
        for poly in polygons:
            all_lats.extend([p[0] for p in poly])
            all_lons.extend([p[1] for p in poly])
        
        center_lat = sum(all_lats) / len(all_lats)
        center_lon = sum(all_lons) / len(all_lons)
        
        # Create map centered on polygon
        m = folium.Map(location=[center_lat, center_lon], zoom_start=15)     
        # Add all polygons to map
        for idx, polygon in enumerate(polygons):
            polygon_coords = [(lat, lon) for lat, lon in polygon]
            popup_text = f'Allowed Area {idx+1}' if len(polygons) > 1 else 'Allowed Area'
            folium.Polygon(
                locations=polygon_coords,
                color='blue',
                fill=True,
                fillColor='blue',
                fillOpacity=0.1,
                popup=popup_text
            ).add_to(m)
        
        # Extract GPS points from JSONL
        inside_points = []
        outside_points = []
        
        with open(output_jsonl, 'r') as f:
            for i, line in enumerate(f):
                record = json.loads(line)
                if record.get("gnss"):
                    first_gnss = next(iter(record["gnss"].values()), None)
                    if first_gnss and "data" in first_gnss:
                        lat = first_gnss["data"]["latitude"]
                        lon = first_gnss["data"]["longitude"]
                        
                        if record.get("in_area"):
                            inside_points.append([lat, lon])
                        else:
                            outside_points.append([lat, lon])
        
        # Add GPS points as circles (display only every 10th point)
        for i, (lat, lon) in enumerate(inside_points):
            if i % 10 == 0:  # Only display every 10th point
                folium.CircleMarker(
                    location=[lat, lon],
                    radius=2,
                    color='green',
                    fill=True,
                    fillOpacity=0.6
                ).add_to(m)

        for i, (lat, lon) in enumerate(outside_points):
            if i % 10 == 0:  # Only display every 10th point
                folium.CircleMarker(
                    location=[lat, lon],
                    radius=2,
                    color='red',
                    fill=True,
                    fillOpacity=0.6
                ).add_to(m)
        
        # Add legend
        legend_html = f'''
        <div style="position: fixed; top: 10px; right: 10px; z-index:9999; 
                    background-color: white; padding: 10px; border: 2px solid grey; border-radius: 5px;">
        <p style="margin: 0;"><b>Legend</b></p>
        <p style="margin: 0;"><span style="color: blue;">━</span> Polygon boundary</p>
        <p style="margin: 0;"><span style="color: green;">●</span> Inside ({len(inside_points)} points)</p>
        <p style="margin: 0;"><span style="color: red;">●</span> Outside ({len(outside_points)} points)</p>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))
        
        # Save and display map
        map_path = OUTPUT_DIR / "gps_map.html"
        m.save(str(map_path))
        print(f"✓ Interactive map saved to: {map_path}")
        print(f"  Open in browser to explore")
        
        # Display in notebook
        display(m)
    else:
        print("GPS polygon or JSONL file not available")
        
except ImportError:
    print("⚠ folium not installed. Run: pip install folium")
    print("  Or use the matplotlib visualization above")


GPS polygon or JSONL file not available


In [13]:
# Preview first 3 records from JSONL file
import json

if output_jsonl.exists():
    print(f"Reading from: {output_jsonl}\n")
    with open(output_jsonl, 'r') as f:
        for i, line in enumerate(f):
            if i >= 3:  # Show first 3 records
                break
            record = json.loads(line)
            print(f"Record {i+1}:")
            print(json.dumps(record, indent=2))
            print("\n" + "="*80 + "\n")
    
    # Count total records
    with open(output_jsonl, 'r') as f:
        total = sum(1 for _ in f)
    print(f"Total records in file: {total:,}")
else:
    print(f"Output file not found: {output_jsonl}")


Reading from: /mnt/data/gnss_aligned_data/rosbag2_2025_09_24-14_13_40_aligned.jsonl

Record 1:
{
  "reference_timestamp": 1758716025945350144,
  "images": {
    "/zed/zed_node/left_raw/image_raw_color/compressed": {
      "timestamp": 1758716025921818400,
      "path": null
    }
  },
  "gnss": {
    "/novatel/oem7/fix": {
      "timestamp": 1758716025946286816,
      "data": {
        "latitude": 51.25794017277804,
        "longitude": 12.535126753042976,
        "altitude": 200.46885417960584,
        "status": 0,
        "service": 15
      }
    }
  },
  "tf": {
    "/tf": {
      "timestamp": 1758716025945350048,
      "data": {
        "transforms": [
          {
            "child_frame_id": "zed_camera_link",
            "header": {
              "frame_id": "odom",
              "stamp": {
                "sec": 1758716025,
                "nanosec": 755078664
              }
            },
            "transform": {
              "translation": {
                "x": 0.0,
   

## Batch Processing

Process all rosbags in the directory:


In [14]:
# Process all rosbags (optional - uncomment to run)
"""
succeeded = 0
failed = 0

for rosbag_dir in os.listdir(ROSBAGS_DIR_NAS):
    rosbag_path = os.path.join(ROSBAGS_DIR_NAS, rosbag_dir)
    
    if not os.path.isdir(rosbag_path):
        continue
    
    # Check if metadata.yaml exists
    if not os.path.exists(os.path.join(rosbag_path, "metadata.yaml")):
        print(f"Skipping {rosbag_dir} (no metadata.yaml)")
        continue
    
    # Skip if output already exists
    output_jsonl = OUTPUT_DIR / f"{rosbag_dir}_aligned.jsonl"
    if output_jsonl.exists():
        print(f"Skipping {rosbag_dir} (already processed)")
        continue
    
    try:
        result = process_rosbag_with_data(rosbag_path, output_jsonl)
        
        if result:
            ref_ts, aligned_data, topic_classifications, topic_messages, bag_name = result
            records_written = write_aligned_jsonl(
                ref_ts, aligned_data, topic_classifications, 
                topic_messages, bag_name, output_jsonl
            )
            succeeded += 1
        else:
            failed += 1
    except Exception as e:
        print(f"Exception processing {rosbag_dir}: {e}")
        failed += 1

print(f"\n{'='*80}")
print(f"Batch processing complete!")
print(f"Succeeded: {succeeded}")
print(f"Failed: {failed}")
print(f"Total: {succeeded + failed}")
print(f"{'='*80}")
"""
print("Batch processing code ready (uncomment to run)")


Batch processing code ready (uncomment to run)


## Output Format Example

Each line in the JSONL file will look like this:

```json
{
  "reference_timestamp": 1740743431773208900,
  "images": {
    "/camera/rear_left/compressed": {
      "timestamp": 1740743431773208900,
      "path": "/path/to/extracted_images_per_topic/rosbag_name/__camera__rear_left__compressed/1740743431773208900.webp"
    },
    "/camera/rear_mid/compressed": {
      "timestamp": 1740743431740046707,
      "path": "/path/to/extracted_images_per_topic/rosbag_name/__camera__rear_mid__compressed/1740743431740046707.webp"
    }
  },
  "gnss": {
    "/gnss/fix": {
      "timestamp": 1740743431750000000,
      "data": {
        "latitude": 48.123456,
        "longitude": 11.234567,
        "altitude": 523.45,
        "status": 0,
        "service": 1
      }
    }
  },
  "tf": {
    "/tf": {
      "timestamp": 1740743431760000000,
      "data": {
        "transforms": [{
          "child_frame_id": "base_link",
          "header": {
            "frame_id": "odom",
            "stamp": {"sec": 1740743431, "nanosec": 760000000}
          },
          "transform": {
            "translation": {"x": 1.23, "y": 0.45, "z": 0.0},
            "rotation": {"x": 0.0, "y": 0.0, "z": 0.1, "w": 0.995}
          }
        }]
      }
    }
  }
}
```


In [15]:
import json

# Get only records inside the polygon
inside_area = []
outside_area = []

with open(output_jsonl, 'r') as f:
    for line in f:
        record = json.loads(line)
        if record["in_area"]:
            inside_area.append(record)
        else:
            outside_area.append(record)

print(f"Inside area: {len(inside_area)}")
print(f"Outside area: {len(outside_area)}")


### Get Images from Specific Area:

# Extract image paths for records inside the polygon
image_paths = []

with open(output_jsonl, 'r') as f:
    for line in f:
        record = json.loads(line)
        if record["in_area"] and record["images"]:
            for topic, img_data in record["images"].items():
                if img_data["path"]:
                    image_paths.append(img_data["path"])

print(f"Found {len(image_paths)} images in the allowed area")

Inside area: 0
Outside area: 6605
Found 0 images in the allowed area
